In [ ]:
%pip install nltk

In [ ]:
%pip install wordcloud

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [5]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession \
        .builder \
        .appName("SparkStreaming") \
        .getOrCreate()

In [2]:
def treatment_tweets(df):
    words = df\
        .select(f.explode(f.split(f.lower('_c0'), ' ')).alias('word'))\
        .withColumn('word', f.regexp_replace('word', r'http\S+', ''))\
        .withColumn('word', f.regexp_replace('word', r'@\w+', ''))\
        .withColumn('word', f.regexp_replace('word', r'rt', ''))\
        .na.replace('', None)\
        .na.drop()
    
    return words

In [ ]:
stops = stopwords.words('portuguese')
stops.append('futebol')

plt.figure(figsize=(20,10))

while True:
    try:
        words = spark.read.csv('./csv', encoding='utf-8')
        words = treatment_tweets(words)
        # Transforma df em uma lista de linhas
        rows = words.collect()
        all_words = ''

        for row in rows:
            all_words = all_words + ' ' + row['word']
        
        word_cloud = WordCloud(stopwords=stops,
                                background_color='black',
                                width=1920,
                                height=1080,
                                max_words=100).generate(all_words)

        plt.cla()
        plt.axis('off')
        plt.imshow(word_cloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        # Vai limpar o output de 5 em 5s
        time.sleep(5)
    
    except KeyboardInterrupt:
        break